# 連想配列と連結リスト


## キーポイント

連想配列：

* 連想配列は「どんなデータでも添え字にできる配列」
* 連想配列では、添字のことを「キー」と呼ぶ
* 連想配列はデータの検索・追加・削除がとても速い
* C++の主な連想配列は`unodered_set`型(アンオーダード・セット)と`unordered_map`型(アンオーダード・マップ)の2つ

双方向リスト：

* 双方向リストは、「隣り合うデータ同士をポインタ等で連結したデータ構造」
* 双方向リストは、先頭と末尾のどちらにも高速にデータを追加できる
* 双方向リストの要素を指すイテレータがある場合、その要素を削除したり、その要素の前に新しいデータを追加するのがとても速い
* C++の双方向リストは`list`型


----

## 1 unordered_set(アンオーダード・セット)

----


### 1.1 find関数の比較回数

「文章に特定の単語が含まれているか調べるプログラム」を作ることを考えます。<br>
思いつく方法として、次のようなものがあります。

1. `vector<string>`に単語単位で文章を読み込む
2. `find`関数で単語を探す

この方針で作成したプログラムを次に示します。

In [ ]:
# @title コード
%%writefile find_word_using_find.cpp
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <algorithm>
using namespace std;

int main() {
  vector<string> v;
  v.reserve(1000);

  fstream ifs("sample.txt");
  for (;;) {
    string s;
    ifs >> s;
    if (ifs.eof()) {
      break;
    }
    v.push_back(s);
  }

  string x;
  cin >> x;

  auto i = find(v.begin(), v.end(), x);

  if (i != v.end()) {
    cout << x << " is found." << endl;
  } else {
    cout << x << " is not found." << endl;
  }
}

In [ ]:
# @title 実行テスト
!echo "The names John Doe for males, Jane Doe or Jane Roe for females, or Jonnie Doe and Janie Doe for children, or just Doe non-gender-specifically are used as placeholder names for a party whose true identity is unknown or must be withheld in a legal action, case, or discussion. The names are also used to refer to acorpse or hospital patient whose identity is unknown. This practice is widely used in the United States and Canada, but is rarely used in other English-speaking countries including the United Kingdom itself, from where the use of John Doe in a legal context originates. The names Joe Bloggs or John Smith are used in the UK instead, as well as in Australia and New Zealand.">sample.txt
!echo "John Doe is sometimes used to refer to a typical male in other contexts as well, in a similar manner to John Q. Public, known in Great Britain as Joe Public, John Smith or Joe Bloggs. For example, the first name listed on a form is often John Doe, along with a fictional address or other fictional information to provide an example of how to fill in the form. The name is also used frequently in popular culture, for example in the Frank Capra film Meet John Doe. John Doe was also the name of a 2002 American television series.">>sample.txt
!echo "Similarly, a child or baby whose identity is unknown may be referred to as Baby Doe. A notorious murder case in Kansas City, Missouri, referred to the baby victim as Precious Doe. Other unidentified female murder victims are Cali Doe and Princess Doe. Additional persons may be called James Doe, Judy Doe, etc. However, to avoid possible confusion, if two anonymous or unknown parties are cited in a specific case or action, the surnames Doe and Roe may be used simultaneously; for example, John Doe v. Jane Roe. If several anonymous parties are referenced, they may simply be labelled John Doe #1, John Doe #2, etc. (the U.S. Operation Delego cited 21 (numbered) John Doe s) or labelled with other variants of Doe / Roe / Poe / etc. Other early alternatives such as John Stiles and Richard Miles are now rarely used, and Mary Major has been used in some American federal cases." >> sample.txt
!g++ -std=c++20 -O2 -Wall -Wextra -o find_word_using_find find_word_using_find.cpp && echo "この下をクリックして検索したい単語を入力" && ./find_word_using_find

`find`関数による検索は、基本的にはfor文と同じで、範囲の先頭からひとつずつ順番に、条件に合うかどうかを調べます。<br>
つまり、`find`関数を使った次のプログラムは、

```cpp
auto i = find(v.begin(), v.end(), x);
```

for文を使った次のプログラムと同じです。

```cpp
auto i = v.begin();
for (; i != v.end(); i++) {
  if (*i == x) {
    break;
  }
}
```

`find`関数とfor文に共通の問題は「データ量に比例して検索に時間がかかるようになる」ことです。<br>
データがランダムな位置で見つかると仮定すると、データを見つけるまでに平均して`データ数 / 2`回の比較が行われます。<br>
例えば、単語の数が10万語の文章が入力された場合、平均比較回数は5万回です。

2025年現在のコンピューターであれば、5万回程度の比較は1秒とかからず完了するでしょう。ですが、ゲームのように1/60秒以内に様々な処理を行わなくてはならないアプリや、多くの人から閲覧されて大量のリクエストを受けるWebサイトでは、もっと短い時間で処理を終了させなくてはなりません。

そのためには、`find`やfor文より高速な検索方法を使う必要があります。


### 1.2 連想配列の概要

検索を高速化する方法として、適切なコンテナを使うことが挙げられます。<br>
例えば、「連想配列(れんそうはいれつ)」という種類のコンテナを使うと、かなり少ない比較回数でデータを見つけられます。

連想配列は、簡単に言うと「どんなデータでも添え字にできる配列」です。連想配列の添字は「キー(鍵)」と呼ばれます。<br>
連想配列という名前は、「キーからデータを即座に連想(取得)できる」というイメージから来ています。

連想配列の特徴は以下の2点です。

* どんなデータでも添字(キー)にできる
* データの格納方法が工夫されていて、データの検索・追加・削除がとても速い

>**【キー(鍵)】**<br>
>連想配列では、添え字のことを「キー(鍵)」と呼びます(添え字が「数字」とは限らないため)。<br>
>本テキストでは、キーと書かれていたら「連想配列の添え字」を意味するものとします。

これらの特徴が役立つ場合は連想配列を使い、特にこれらの特徴が不要な場合は`vector`を使うとよいでしょう。

2025年現在、C++の連想配列は全部で8種類あります。

| 名前 | 性質 | 格納する要素 | キーが等しい要素 | 要素の格納方法 | 追加時期 |
|:-----|:-----|:---------------|:------------------------:|:-----------------|:---------|
| <font size=3>unordered_map</font><br>(アンオーダード・マップ) | <font size=2>検索・追加・削除が非常に高速。<br>データの格納順は不定。</font> | キーとデータ | 禁止 | ハッシュ | C++11 |
| <font size=3>unordered_set</font><br>(アンオーダード・セット) | &emsp;(同上) | キー | 禁止 | ハッシュ | C++11 |
| <font size=3>unordered_multimap</font><br>(アンオーダード・マルチ・マップ) | &emsp;(同上) | キーとデータ| 許可 | ハッシュ | C++11 |
| <font size=3>unordered_multiset</font><br>(アンオーダード・マルチ・セット) | &emsp;(同上) | キー | 許可 | ハッシュ | C++11 |
| <font size=3>map</font>(マップ) | <font size=2>検索・追加・削除が高速。<br>データの格納順は一定。</font> | キーとデータ | 禁止 | 平衡二分木 | 初期から存在 |
| <font size=3>set</font>(セット) | &emsp;(同上) | キー | 禁止 | 平衡二分木 | 初期から存在 |
| <font size=3>multimap</font>(マルチ・マップ) | &emsp;(同上) | キーとデータ | 許可 | 平衡二分木 | 初期から存在 |
| <font size=3>multiset</font>(マルチ・セット) | &emsp;(同上) | キー | 許可 | 平衡二分木 | 初期から存在 |

これらの連想配列型は、「初期からある型」と、C++11で追加された「名前に`unordered`(アンオーダード)が付く型」に大別できます。<br>
一般的には、後から追加された「名前に`unordered`が付く型」のほうが性能が良いです。<br>
さらに、「名前に`multi`が付く型」は利用頻度が低めです。

そこで、本テキストでは性能も利用頻度も高めの`unordered_set`型と`unordered_map`型の2つに絞って解説します。


### 1.3 unordered_setの概要

`unordered_set`型は、「キーだけを持つ連想配列」です。「データをキーにしたい」場合に使用します。<br>
`unorderd_set`というヘッダファイルをインクルードすることで使用できます。

`unordered_set`型は、テンプレートを使って次のように宣言されています。

```cpp
template<typename Key>
class unordered_set;
```

>`class`(クラス)キーワードを使うと、`struct`キーワードと同じように型を宣言できます。詳細は回を改めて説明します。

このように、「テンプレートを使って宣言された型」のことを「クラス・テンプレート」といいます。

さて、少し上で書いた`vector`型を使ったプログラムを、`unordered_set`型を使って置き換えると、次のようになります。<br>
以下のコードを`▶`ボタンをクリックして保存し、下にある「実行テスト」セルの`▶`ボタンをクリックして、動作を確認してください。


In [ ]:
# @title コード
%%writefile find_word_using_unordered_set.cpp
#include <iostream>
#include <fstream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> v;
  v.reserve(10'000);

 fstream ifs("sample.txt");
  for (;;) {
    string s;
    ifs >> s;
    if (ifs.eof()) {
      break;
    }
    v.emplace(s);
  }

  string x;
  cin >> x;

  auto i = v.find(x);

  if (i != v.end()) {
    cout << "Yes" << endl;
  } else {
    cout << "No" << endl;
  }
}


In [ ]:
# @title 実行テスト
!echo "One way to speed up searches is to use an appropriate container .">sample.txt
!echo "For example , if you use a type of container called an Associative Array , you can find data with a significantly reduced number of comparisons .">>sample.txt
!echo "Simply put , an associative array is an array in which any data can be used as a subscript .">>sample.txt
!echo "The subscripts in an associative array are called Key .">>sample.txt
!echo "The name associative array comes from the idea that data can be instantly associated ( retrieved ) from a key .">>sample.txt
!g++ -std=c++20 -O2 -Wall -Wextra -o find_word_using_unordered_set find_word_using_unordered_set.cpp && echo "この下をクリックして検索したい単語を入力" && ./find_word_using_unordered_set

`unordered_set`にデータを追加するには、`emplace`(エンプレイス)メンバ関数を使います。<br>
データの検索には、`unordered_set`自身の`find`メンバ関数を使います(汎用アルゴリズムの`find`関数は使いません)。

`unordered_set`の`find`メンバ関数は`unordered_set`専用の特別な設計になっていて、どれほどデータ数が多くなっても、ほぼ1回の比較で目的のデータを見つけられます。


### 1.4 キーの有無を調べる

`unordered_set`に「特定のキー」が存在することを調べるには、`find`(ファインド)メンバ関数を使います。

>**書式**
>
>```cpp
>iterator find(const Key& key);
>```
>
>**引数**
>
>* key&emsp;検索するキー
>
>**戻り値**
>
>引数`key`と一致するキーが見つかれば、そのキーを差すイテレータを返します。<br>
>見つからなければ、`end`メンバ関数の戻り値を返します。

`find`関数の戻り値は双方向イテレータです。イテレータを「参照外し」することで、データを読み書きできます。

もしキーが見つからなかった場合、戻り値のイテレータは、`unordered_set`の`end`メンバ関数の値と一致します。

次のプログラムは、`find`メンバ関数の使用例です。

**コード**

```cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> v = { "andy", "barbie", "conrad", "davis", "elli", "finn" };

  auto i = v.find("gaia");
  if (i != v.end()) {
    cout << *i << "はあります" << endl;
  } else {
    cout << "gaiaはありません" << endl;
  }

  i = v.find("conrad");
  if (i != v.end()) {
    cout << *i << "はあります" << endl;
  } else {
    cout << "conradはありません" << endl;
  }
}
```

**実行結果**

```txt
gaiaはありません
conradはあります
```



### 1.5 キーを追加する

`unordered_set`にキーを追加するには、`emplace`(エンプレイス)メンバ関数を使います。

>**書式**
>
>```cpp
>pair<iterator, bool> emplace(const Key& key);
>```
>
>**引数**
>
>* key&emsp;追加するキー
>
>**戻り値**
>
>引数`key`と一致するキーがなければ、追加したキーを差すイテレータと`true`のペアを返します。<br>
>一致するキーがあれば、そのキーを指すイテレータと`false`のペアを返します。

このメンバ関数の戻り値は「追加したデータの位置をあらわすイテレータ」と、「追加の結果をあらわす真偽値」のペアです。


#### pair(ペア)構造体

`emplace`メンバ関数は2つの値を返す必要があるため、標準ライブラリにある`pair`(ペア)型を使っています。<br>
`pair`型は次のような、「テンプレート引数によって、2つのメンバ変数の型を指定できる構造体」として定義されています。

```cpp
template<typename T, typename U>
struct pair {
  T  first;  // １個目のデータ
  U  second; // ２個目のデータ
};
```

例えば、`pair<iterator, bool>`型は次のような構造体として扱われます。

```cpp
struct pair<iterator, bool> {
  iterator first;
  bool     second;
};
```


`emplace`関数の戻り値の`pair`構造体は、キーの追加に成功したか失敗したかで、次のように変化します。

| 追加の成否 | firstの内容 | secondの内容 |
|:----:|:------|:-------|
| 成功 | 追加したキーを指すイテレータ | true |
| 失敗 | 追加しようとしたキーと同じキーを指すイテレータ | false |

追加の成否は、`second`メンバ変数を調べることで確認できます。

次のプログラムは、`empalce`メンバ関数の使用例です。

**コード**

```cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> v = { "andy", "barbie", "conrad", "davis", "elli", "finn" };

  // キーがgaiaの要素はないので成功する(secondにtrueが代入される)
  auto a = v.emplace("gaia");
  if (a.second) {
    cout << *a.first << "を追加" << endl;
  } else {
    cout << "gaiaの追加に失敗" << endl;
  }

  // キーがdavisの要素はすでにあるので失敗する(secondにfalseが代入される)
  auto b = v.emplace("davis");
  if (b.second) {
    cout << *a.first << "を追加" << endl;
  } else {
    cout << "davisの追加に失敗" << endl;
  }
}
```

**実行結果**

```txt
gaiaを追加
davisの追加に失敗
```

>なお、`pair`構造体は、連想配列以外の場面でも普通に使えます。2つの値をまとめて扱いたい場合に便利です。


### 1.6 キーを削除する

`unordered_set`からデータを削除するには、`erase`(イレース)メンバ関数を使います。<br>
設計上の理由から、汎用アルゴリズムの`erase`は連想配列に対応していないため、間違って使用するとコンパイルエラーになります。<br>
注意してください(ただし、`erase_if`汎用アルゴリズムは使えます)。

>**書式**
>
>```cpp
>size_type erase(const Key& key);
>```
>
>**引数**
>
>* key&emsp;削除するキー
>
>**戻り値**
>
>引数`key`と一致するキーがなければ`0`を返します。<br>
>一致するキーがあれば`1`を返します。

`erase`メンバ関数の戻り値は「削除したキーの数」です。この値は、削除に成功したら`1`、失敗したら`0`になります。

>**【size_typeについて】**<br>
>`size_type`(サイズ・タイプ)型は、コンテナ(ここでは`unordered_set`)ごとに宣言されている「大きさをあらわす型」です。ほぼすべての環境で`size_t`型の別名なので、`size_t`型だと考えて差し支えありません。

次のプログラムは、`erase`メンバ関数の使用例です。

**コード**

```cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> v = { "andy", "barbie", "conrad", "davis", "elli", "finn" };

  if (v.erase("andy")) {
    cout << "andyを削除" << endl;
  } else {
    cout << "andyを削除できません" << endl;
  }

  auto i = v.find("andy");
  if (i == v.end()) {
    cout << "andyはありません" << endl;
  } else {
    cout << "andyはあります" << endl;
  }
}
```

**実行結果**

```txt
andyを削除
andyはありません
```


### 1.7 連想配列全体を走査する

`unordered_set`が保持するすべてのデータを調べるには、`begin`関数と`end`関数が返すイテレータを使います。

**コード**

```cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> v = { "andy", "barbie", "conrad", "davis", "elli", "finn" };

  // すべての要素を表示
  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << ' ';
  }
  cout << endl;
}
```

**実行結果**

```txt
finn elli davis conrad barbie andy
```

`begin`関数と`end`関数を使う方法は、このあとで解説する`unorered_map`型や`list`型でも同様に使えます。


### 1.8 連想配列の要素数を調べる

連想配列に記録されている要素数を調べるには、`size`(サイズ)メンバ関数を使います。

**コード**

```cpp
#include <iostream>
#include <unordered_set>
using namespace std;

int main() {
  unordered_set<int> a = { 1, 2, 3, 5 };
  cout << a.size() << endl;

  a.emplace(8);
  cout << a.size() << endl;

  a.erase(5);
  a.erase(3);
  cout << a.size() << endl;
}
```

**実行結果**

```txt
4
5
3
```



----

## 2 unordered_map(アンオーダード・マップ)

----


### 2.1 unordered_mapの概要

`unordered_map`型は、「キーとデータをペアにして持つ連想配列」です。「キーとデータが異なる」場合に使用します。<br>
`unordered_map`というヘッダファイルをインクルードすることで使用できます。

`unordered_map`型は、テンプレートを使って次のように宣言されています。

```cpp
template<typename Key, typename Value>
class unordered_map;
```

`unordered_map`が使われるのは、例えば「名前に対応するデータを検索したい」場合です。<br>
次のプログラムは、`unordered_map`を使って、「名前」と「得点」を関連付ける例です。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  // []演算子を使ってデータを読み書きできる
  string s = "barbie";
  cout << s << "の得点は" << v[s] << "点です" << endl;
}
```

**実行結果**

```txt
barbieの得点は75点です
```

上記のプログラムで、初期化の`{}`の中身が、キーとデータのペアになっている点に注目してください。<br>
例えば、最初のペアは`{"andy", 60}`で、キーは`"andy"`、データは`60`となります。

また、`unordered_map`は`[]`演算子にキーを指定することで、キーに対応するデータを読み書きできます。


### 2.2 unorderd_mapとpair構造体

連想配列は、キーとデータをペアにして格納します。これは、標準ライブラリの`pair`(ペア)構造体によって実現しています。

`pair`構造体は次のように、テンプレートとして定義されています。

```cpp
template<typename T, typename U>
struct pair {
  T  first;  // １個目のデータ
  U  second; // ２個目のデータ
};
```

例えば、`unordered_map<char, string>`型が使用するペアは、次のようになります。

```cpp
struct pair<char, string> {
  char   first;
  string second;
};
```

キーは`first`(ファースト)メンバ変数に格納されます。<br>
データは`second`(セカンド)メンバ変数に格納されます。


### 2.3 キーの有無を調べる

連想配列は何でもキーにできるため、「キーに対応するデータがない」場合のことも考えなくてはなりません。

例えば、すぐ上のプログラムでキーを`barbie`から`gaia`に変えたとします。しかし、連想配列の中にキーが`gaia`のデータはありません。<br>
この場合、`unordered_map`は、指定されたキーにデフォルト値を割り当てます。`int`などの数値型のデフォルト値は`0`です。

**コード**

```cpp
int main() {
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  // []演算子を使ってデータを読み書きできる
  string s = "gaia"; // barbieをgaiaに変更
  cout << s << "の得点は" << v[s] << "点です" << endl;
}
```

**実行結果**

```txt
gaiaの得点は0点です
```

そのため、上記の`v[s]`というプログラムは新たに「文字列`gaia`と整数`0`のペア」を作成します。<br>
そして、`v[s]`の結果として`0`を返します。ですが、これは意図した挙動ではないでしょう。<br>
大抵は、`gaia`が`v`に含まれないことを判定して、「`gaia`という名前は登録されていません」などと出力するほうが適切です。

あるキーが存在することを調べるには、`find`(ファインド)メンバ関数を使います。

>**書式**
>
>```cpp
>iterator find(const Key& key);
>```
>
>**引数**
>
>* key&emsp;検索するキー
>
>**戻り値**
>
>引数`key`と一致するキーが見つかれば、そのキーを差すイテレータを返します。<br>
>見つからなければ、`end`メンバ関数の戻り値を返します。

`find`関数の戻り値は双方向イテレータです。<br>
`unrodered_set`と異なり、`unordered_map`のイテレータは「キーとデータのペア」の位置をあらわします。<br>
そのため、`->`演算子を使ってペアのメンバ変数を読み書きします。

もしキーが見つからなかった場合、戻り値のイテレータは`unordered_map`の`end`メンバ関数の値と一致します。

次のプログラムは、`find`メンバ関数の使用例です。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  // 名前と得点をペアとして記録する連想配列
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  string s = "gaia";
  auto i = v.find(s);

  if (i != v.end()) {
    cout << i->first << "の得点は" << i->second << "点です" << endl;
  } else {
    cout << s << "という名前は登録されていません" << endl;
  }

  s = "elli";
  i = v.find(s);

  if (i != v.end()) {
    cout << i->first << "の得点は" << i->second << "点です" << endl;
  } else {
    cout << s << "という名前は登録されていません" << endl;
  }
}
```

**実行結果**

```txt
gaiaという名前は登録されていません
elliの得点は90点です
```


### 2.4 データの読み書き

`[]`(添え字)演算子の添え字としてキーを指定すると、キーに関連付けられたデータを読み書きできます。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  cout << "andyの現在の得点 : " << v["andy"] << endl;

  v["andy"] = 70;

  cout << "andyの新しい得点 : " << v["andy"] << endl;
}
```

**実行結果**

```txt
andyの現在の得点 : 60
andyの新しい得点 : 70
```


### 2.5 キーとデータを追加する

`unordered_map`にデータを追加するには`try_emplace`(トライ・エンプレイス)メンバ関数を使います。

>**書式**
>
>```cpp
>pair<iterator, bool> try_emplace(const Key& key, const Vaule& value);
>```
>
>**引数**
>
>* key&emsp;&nbsp;&nbsp;&nbsp;追加するキー
>* value&emsp;追加するデータ
>
>**戻り値**
>
>引数`key`と一致するキーがなければ、追加したキーを差すイテレータと`true`のペアを返します。<br>
>一致するキーがあれば、そのキーを指すイテレータと`false`のペアを返します。

`try_emplace`メンバ関数の戻り値は、「追加したキーの位置をあらわすイテレータ」と、「追加の結果をあらわす真偽値」のペアです。<br>
ペアの値は、キーの追加に成功したか失敗したかで、次のように変化します。

| 追加の成否 | firstの内容 | secondの内容 |
|:----:|:------|:-------|
| 成功 | 追加したキーを指すイテレータ | true |
| 失敗 | 追加しようとしたキーと同じキーを指すイテレータ | false |

追加の成否は、`second`メンバ変数を調べることで確認できます。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  // キーがgaiaの要素はないので成功する
  auto a = v.try_emplace("gaia", 40);
  if (a.second) {
    auto i = a.first; // イテレータを取得
    cout << i->second << "の得点を追加" << endl;
  } else {
    cout << "gaiaの追加に失敗" << endl;
  }

  // キーがgaiaの要素はすでにあるので失敗する
  auto b = v.try_emplace("gaia", 50);
  if (b.second) {
    cout << b.first->second << "を追加" << endl;
  } else {
    cout << "gaiaの追加に失敗" << endl;
  }

  // キーがgaiaの要素を検索
  auto i = v.find("gaia");
  if (i == v.end()) {
    cout << i->first << "の得点: " << i->second << endl;
  } else {
    cout << "gaiaという名前は登録されていません" << endl;
  }
}
```

**実行結果**

```txt
gaiaの得点を追加
gaiaの追加に失敗
gaiaの得点: 40
```


### 2.6 []演算子でキーとデータを追加する

あまり推奨されませんが、`[]`演算子を使うことでも、キーとデータを追加できます。

`[]`演算子には、

&emsp;**キーが存在しない場合は自動的にキーを追加し、「空のデータ」を設定する**

という機能があるからです。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v;

  auto i = v.find("andy");
  if (i != v.end()) {
    cout << "andyは登録済みです(得点:" << i->second << ")" << endl;
  } else {
    cout << "andyは未登録です" << endl;
  }

  v["andy"] = 60;
  cout << "andyを登録" << endl;

  i = v.find("andy");
  if (i != v.end()) {
    cout << "andyは登録済みです(得点:" << i->second << ")" << endl;
  } else {
    cout << "andyは未登録です" << endl;
  }
}
```

**実行結果**

```txt
andyは未登録です
andyを登録
andyは登録済みです(得点:60)
```

つまり、「`[]`演算子は追加と更新の両方の機能を持つ」わけです。<br>
便利な機能ですが、「更新したいときに間違ったキーを指定してしまうと、新しいキーが作られてしまう」という問題もあります。

そのため、データを更新するとき、キーが確実に存在すると分かっているなら`[]`演算子を使い、不明な場合は`find`メンバ関数を使って、キーの有無を確認するとよいでしょう。

また、データを追加するときは、`[]`演算子より`try_emplace`メンバ関数を優先してください。<br>
`[]`演算子では、「追加したい」のか「更新したい」のかが分かりにくいためです。<br>
その点、`try_empalce`は、見ただけで「追加したい」ということが分かります。

### 2.7 キーとデータを削除する

`unordered_map`からキーとデータを削除するには`erase`(イレース)メンバ関数を使います。<br>
なお、設計上の理由から、汎用アルゴリズムの`erase`は連想配列に対応していないので使えないので、注意してください<br>
(ただし、`erase_if`汎用アルゴリズムは使えます)。

>**書式**
>
>```cpp
>size_type erase(const Key& key);
>```
>
>**引数**
>
>* key&emsp;削除するキー
>
>**戻り値**
>
>引数`key`と一致するキーがなければ`0`を返します。<br>
>一致するキーがあれば`1`を返します。

`erase`メンバ関数の戻り値は「削除したペアの数」です。この値は、削除に成功したら`1`、失敗したら`0`になります。

次のプログラムは、`erase`メンバ関数の使用例です。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  auto i = v.find("conrad");
  if (i == v.end()) {
    cout << "conradは登録されていません" << endl;
  } else {
    cout << i->first << "の得点: " << i->second << endl;
  }

  if (v.erase("conrad")) {
    cout << "conradを削除" << endl;
  } else {
    cout << "conradを削除できません" << endl;
  }

  i = v.find("conrad");
  if (i == v.end()) {
    cout << "conradは登録されていません" << endl;
  } else {
    cout << i->first << "の得点: " << i->second << endl;
  }
}
```

**実行結果**

```txt
conradの得点: 57
conradを削除
conradは登録されていません
```



### 2.8 連想配列を「空きの多い配列」として使う

>ここは少し難しい話です。今はまだ分からなくても、あまり気にしないでください。

$100m^2$ の平面空間に、100体の敵がいて、敵同士が重ならないようにしたいとします。

敵同士の衝突判定は、1体につき残り99体の敵との衝突判定を行うことで実現できます。この方法では、9900回の衝突判定を行う必要があります。

しかし、「衝突するほど近い距離にいる敵の数」は、多くても10体くらいでしょう。<br>
ということは、「衝突するほど近い敵」を判別できれば、衝突判定の回数を、1000回程度まで減らせるはずです。

このような場合、例えば $100m^2$ 空間を $5m^2$ 単位に分割して、20x20マスの配列を作ります。すべての敵を「位置に対応するマス」に登録します。<br>
このデータ構造では、同じマスにいる敵は「衝突するほど近い敵」と判定できます。

**配列の例(4x4マスに8体の敵がいる)**

```txt
   0   5  10  15  20
 0┌─┬─┬─┬─┐    v[0]  = { 敵1, 敵2 }
  │ 0│ 1│ 2│ 3│    v[1]  = {}
 5├─┼─┼─┼─┤    v[2]  = { 敵6 }
  │ 4│ 5│ 6│ 7│    v[3]  = {}
10├─┼─┼─┼─┤    v[4]  = { 敵4, 敵8 }
  │ 8│ 9│10│11│    v[5]  = {}
15├─┼─┼─┼─┤    v[6]  = {}
  │12│13│14│15│    v[7]  = {}
20└─┴─┴─┴─┘    v[8]  = {}
                        v[9]  = {}
                        v[10] = { 敵3 }
                        v[11] = {}
                        v[12] = {}
                        v[13] = {}
                        v[14] = {}
                        v[15] = { 敵5, 敵7 }
```

**コード**

```cpp
#include <iostream>
#include <vector>
using namespace std;

struct Enemy {
  bool HitCheck(const Enemy& other) {
    return abs(x - other.x) <= 1 && abs(y - other.y) <= 1;
  }

  int ToIndex() { return (y / 5) * 4 + (x / 5); }

  int x, y;
};

int main() {
  // 敵の配列
  vector<Enemy> enemies = { { 2, 3}, { 4, 1}, {13,13}, { 3, 7},
                            {16,17}, {11, 2}, {18,16}, { 3, 8} };
  // 敵をマス目に登録
  vector<vector<Enemy*>> v(16);
  for (int i = 0; i < (int)enemies.size(); i++) {
    int index = enemies[i].ToIndex();
    v[index] = &enemies[i];
  }

  // マス目の敵の衝突判定
  for (int i = 0; i < (int)v.size(); i++) {
    for (int a = 0; a < (int)v[i].size() - 1; a++) {
      Enemy* pa = v[i][a];
      for (int b = a + 1; b < (int)v[i].size(); b++) {
        Enemy* pb = v[i][b];
        if (pa->HitCheck(*pb)) {
          cout << pa->x << ',' << pa->y << ' ' << pb->x << ',' << pb->y << endl;
        }
      } // for b
    } // for a
  } // for i
}
```

**出力結果**

```txt
3,7 3,8
```

>この例では省略していますが、本来、面積を持つ物体の衝突判定では、周囲のマスの敵とも衝突判定を行う必要があります。

２次元配列を使う場合の問題点は、「敵のいないマスのメモリが無駄になる」ということです。<br>
例えば、上の図のように16マス中5マスしか敵がいない場合、11マスは全く使われません。

このような、「空きが多い配列」を効率的に表現するには、連想配列を使います。<br>
例えば、敵のいるマスの番号をキーとする連想配列を作ります。そして、敵のいるマスだけを連想配列に追加します。

**連想配列の例(4x4マスに8体の敵がいる)**

```txt
   0   5  10  15  20
 0┌─┬─┬─┬─┐    m[0]  = { 敵1, 敵2 }
  │ 0│ 1│ 2│ 3│    m[2]  = { 敵6 }
 5├─┼─┼─┼─┤    m[4]  = { 敵4, 敵8 }
  │ 4│ 5│ 6│ 7│    m[10] = { 敵3 }
10├─┼─┼─┼─┤    m[15] = { 敵5, 敵7 }
  │ 8│ 9│10│11│
15├─┼─┼─┼─┤
  │12│13│14│15│
20└─┴─┴─┴─┘
```

**コード**

```cpp
#include <iostream>
#include <vector>
#include <unordered_map>
using namespace std;

struct Enemy {
  bool HitCheck(const Enemy& other) {
    return abs(x - other.x) <= 1 && abs(y - other.y) <= 1;
  }

  int ToIndex() { return (y / 5) * 4 + (x / 5); }

  int x, y;
};

int main() {
  // 敵の配列
  vector<Enemy> enemies = { { 2, 3}, { 4, 1}, {13,13}, { 3, 7},
                            {16,17}, {11, 2}, {18,16}, { 3, 8} };
  // 敵をマス目に登録
  unordered_map<int, vector<Enemy*>> m;
  for (int i = 0; i < (int)enemies.size(); i++) {
    int index = enemies[i].ToIndex();
    m[index].push_back(&enemies[i]);
  }

  // マス目の敵の衝突判定
  for (auto i = m.begin(); i != m.end(); i++) {
    for (int a = 0; a < (int)i->second.size() - 1; a++) {
      Enemy* pa = i->second[a];
      for (int b = a + 1; b < (int)i->second.size(); b++) {
        Enemy* pb = i->second[b];
        if (pa->HitCheck(*pb)) {
          cout << pa->x << ',' << pa->y << ' ' << pb->x << ',' << pb->y << endl;
        }
      } // for b
    } // for a
  } // for i
}
```

**出力結果**

```txt
3,7 3,8
```

連想配列に追加されるマスは0, 2, 4, 10, 15の5つだけなので、敵のいないマスのために無駄なメモリを使わなくて済みます。

>**【ひとつのキーに複数のデータを割り当てる他の方法】**<br>
>連想配列の例では、複数のデータを割り当てるために配列を使いました。<br>
>他の方法として、`unordered_multimap`(アンオーダード・マルチマップ)というクラスを使うこともできます。<br>
>名前に「マルチ」とあるように、アンオーダード・マルチマップは、同じキーに対して複数のデータを割り当てられます。

>**【キーとデータが同じ場合】**<br>
>キーとデータが等しい、つまりキーだけで十分な場合、`unordered_set`(アンオーダード・セット)クラスを使います。<br>
>アンオーダード・セットはキーしか持たない連想配列です。



----

## 3 list(リスト)

----


### 2.1 連結リストの概要

「連結(れんけつ)リスト」は、「隣り合うデータ同士をポインタ等で連結した」データ構造で、次の特徴を持ちます。

* 追加、または削除する要素の位置が分かっている場合、どの位置であっても非常に高速に追加や削除ができる
* 添え字演算子(`[]`)が使えない
* イテレータの種類が「双方向イテレータ」のため、`++`または`--`演算子でしか位置を変えられない。

C++では、連結リストを`list`型として定義しています。

```cpp
template<typename T>
class list;
```

`list`型を使うには、`list`というヘッダファイルをインクルードします。<br>
次のプログラムは、`list`型の変数に対して要素を削除、追加する例です。

**コード**

```cpp
#include <iostream>
#include <list>
using namespace std;

int main() {
  list<int> v = { 1, 2, 3, 4, 5 };

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << " ";
  }
  cout << endl;

  // 先頭から2番目の要素を削除
  auto i = v.begin();
  i++;
  i++;
  v.erase(i);

  // 末尾の一つ前に10を追加
  i = v.end();
  i--;
  v.emplace(i, 10);

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << " ";
  }
  cout << endl;
}
```

**実行結果**

```txt
1 2 3 4 5
1 2 4 10 5
```


### 2.2 連結リストのデータ構造

`list`の各要素は、以下のような要素単位のメモが、メモ帳のあちこちに散らばっていると考えられます。

```txt
前の要素の位置: ◯ページ△行目
次の要素の位置: □ページ✕行目
データ: 123
```

データの型をテンプレート引数`T`(ティー)とすると、このメモは次のような構造体としてあらわせます。

```cpp
template<typename T>
struct ListData {
  ListData* prev; // 前の要素の位置
  ListData* next; // 次の要素の位置
  T data;         // データ
};
```

実際の`list`型の要素とは細部が異なりますが、おおよそはこの形をしていると考えてください。<br>
この種類のデータ構造が「連結リスト」と呼ばれるのは、`ListData`同士が、次の図のように互いに連結されているからです。

$$
list-\boxed{ListData}-\boxed{ListData}-\boxed{ListData}-...
$$

このように、連結リストの構造は、`vector`のような配列と比べて複雑です。<br>
そのため、格納されたデータを読み書きする、という基本的な用途だけを考えると、`vector`よりも性能が低くなります。



### 2.3 vectorとlistのどちらを使うべきか

基本的な考え方は「とりあえず`vector`を使おう」です。

#### listの利点

&emsp;先頭や中間部分の追加や削除は、`list`のほうが`vector`よりかなり高速です。<br>
&emsp;中間部分への追加や削除が多い場合は、`list`の使用を検討してください。

&emsp;さらに、`list`は「他の`list`から要素を移動」したり、「2つのリストをまとめる」ような操作も非常に高速です。<br>
&emsp;要素をつなぎ直すだけで済むからです。

#### listの欠点

&emsp;データを末尾に追加したり、末尾のデータを削除する処理は`vector`のほうが高速です。<br>
&emsp;このため、末尾への追加と削除しかしないなら、`list`より`vector`を使うべきです。

&emsp;別の欠点として、`ListData`は隣の要素の位置しか分からない、という事が挙げられます。<br>
&emsp;そのため、「N個先の要素」は、要素をひとつずつ辿っていく以外に知る方法はありません。<br>
&emsp;`list`型が`[]`演算子を使えず、イテレータの種類が双方向イテレータになっているのは、このデータ構造が原因です。

#### listよりvectorを使おう

&emsp;利点と欠点をまとめると、`list`が役に立つのは「リストの途中への追加や削除が頻繁に行われる場合」に限られます。<br>
&emsp;ですが、そういったケースは多くありません。そのため、基本的には`vector`を使うべき、となります。


### 2.4 データを検索する

`list`のデータを検索するには、汎用アルゴリズムの`find`関数を使います。

**コード**

```cpp
#include <iostream>
#include <list>
#include <algorithm>
using namespace std;

int main() {
  list<int> v = { 1, 2, 3, 4, 5 };

  auto i = find(v.begin(), v.end(), 3);
  if (i != v.end()) {
    cout << *i << "が見つかりました" << endl;
  } else {
    cout << "見つかりません" << endl;
  }
}
```

**実行結果**

```txt
3が見つかりました
```


### 2.5 データを追加する

`list`にデータを追加するには、位置に応じて以下の3つのメンバ関数を使い分けます。

>**書式**
>
>```cpp
>void push_back(const T& x);
>void push_front(const T& x);
>iterator insert(iterator i, const T& x);
>```
>
>**引数**
>
>* x&emsp;追加するデータ
>* i&emsp;追加する位置を指すイテレータ
>
>**戻り値**
>
>`emplace`メンバ関数は、追加したデータを指すイテレータを返します。

データを末尾に追加する場合は`push_back`(プッシュ・バック)を使います。<br>
データを先頭に追加する場合は`push_front`(プッシュ・フロント)を使います。<br>
データを好きな位置に追加したい場合は`insert`(インサート)を使います。`insert`は、位置`i`の直前にデータ`x`を追加します。

次のプログラムは、３つの関数を使ってデータを追加する例です。

**コード**

```cpp
#include <iostream>
#include <list>
using namespace std;

int main() {
  list<int> v = { 1, 2, 3, 4, 5 };

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << " ";
  }
  cout << endl;

  // 末尾にデータを追加
  v.push_back(10);

  // 先頭にデータを追加
  v.push_front(20);

  // 先頭から2番目にデータを追加
  auto i = v.begin();
  i++;
  i++;
  v.insert(i, 30);

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << " ";
  }
  cout << endl;
}
```

**実行結果**

```txt
1 2 3 4 5
20 1 30 2 3 4 5 10
```


### 2.6 データを削除する

`list`からデータを削除するには、用途に応じて次のメンバ関数を使い分けます。

>**書式**
>
>```cpp
>void pop_back();<br>
>void pop_front();<br>
>
>iterator erase(iterator i);
>
>size_type remove(const T& x);
>
>template<typename P>
>size_type remove_if(P pred);
>```
>
>**引数**
>
>* i&emsp;削除する要素を指すイテレータ
>* x&emsp;削除するデータ
>* pred&emsp;削除する条件をあらわす関数
>
>**戻り値**
>
>`erase`メンバ関数は、「削除したデータの次のデータ」を指すイテレータを返します。<br>
>`remove`メンバ関数および`remove_if`メンバ関数は、「削除したデータの数」を返します。

`pop_back`(ポップ・バック)メンバ関数は、リストの末尾にある要素を削除します。<br>
`pop_front`(ポップ・フロント)メンバ関数は、リストの先頭にある要素を削除します。

`erase`(イレース)メンバ関数は、引数`i`で指定された位置の要素を削除します。

`remove`(リムーブ)メンバ関数は、引数`x`と一致するすべての要素を削除します。<br>
`remove_if`(リムーブ・イフ)メンバ関数は、引数で指定された関数`pred`が`true`を返す、すべての要素を削除します。

次のプログラムは、3つのメンバ関数を使ってデータを削除する例です。

**コード**

```cpp
#include <iostream>
#include <list>
using namespace std;

void print(const list<int>& v) {
  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << " ";
  }
  cout << endl;
}

int main() {
  list<int> v = { 1, 2, 3, 4, 5, 4, 3, 2 };
  cout << "削除前" << endl;
  print(v);

  auto i = find(v.begin(), v.end(), 2);
  auto j = v.erase(i);
  cout << "削除したデータの次のデータは:" << *j << endl;
  print(v);

  size_t a = v.remove(4);
  cout << "4を" << a << "個削除" << endl;
  print(v);

  size_t b = v.remove_if([](int n){ return n % 2; });
  cout << "奇数を" << b << "個削除" << endl;
  print(v);
}
```

**実行結果**

```txt
削除前
1 2 3 4 5 4 3 2
削除したデータの次のデータは:3
1 3 4 5 4 3 2
4を2個削除
1 3 5 3 2
奇数を4個削除
2
```


### 2.7 データを操作するメンバ関数について

`list`型は複雑な構造なので、コンテナを変更するような操作はメンバ関数として用意されています。

| メンバ関数名 | 機能 |
|:-------------|:-----|
| <font size=3>splice</font>(スプライス) | 他のコンテナから要素を移動する |
| <font size=3>unique</font>(ユニーク) | 重複した要素をコンテナから削除する |
| <font size=3>merge</font>(マージ) | 2つのコンテナを併合する |
| <font size=3>sort</font>(ソート) | コンテナを並べ替える |
| <font size=3>reverse</font>(リバース) | コンテナを反転する |

すべてを説明している余裕はないので、ここでは「こういう機能がある」という紹介にとどめます。<br>
詳しくは`cpprefjp.github.io`などのリファレンスを参照してください。

実際のところ、これらの操作が必要となることは多くありません。<br>
ですが、「`list`型はメンバ関数を使う操作が多い」ということは覚えておくとよいでしょう。


----

## 4 練習問題

----

以下の手順にしたがって、各問題のプログラムを完成させなさい。

1. `%%writefile ...`の下の行からがプログラムです。問題文に従ってプログラムを修正、または追加してください。
2. プログラムを修正したら、セルの右側にある`▶`をクリックします。すると、ファイルが保存されます。
3. 「動作テスト」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、実行結果が表示されます。<br>
   このセルは、修正したプログラムの動作を確認するために使ってください。
4. 「実行」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、結果の成否が判定されます。
5. 判定に成功したら`AC`と表示されます。次の問題に進んでください。
6. 失敗したら`WA`と表示されます(その前にエラーメッセージが表示される場合もあります)。<br>
   これは、プログラムのどこかにエラーがあることを意味します。<br>
   「動作テスト」を使ってエラーを修正し、`AC`を目指してください。


### ❓️問題１ 名前の検索

英国の騎士物語の登場人物を、検索するプログラムを作りたいです。

検索文字列Sが入力されます。<br>
`unordered_set`型の変数`data`に、Sが含まれる場合は`"Yes"`、含まれない場合は`"No"`を出力するプログラムを作成しなさい。

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `auto`型の変数`itr`を宣言し、`find`関数を使って`data`から`s`を検索した戻り値で、変数`itr`を初期化する
2. if文を使って、変数`itr`が`data.end()`と異なる場合、`cout`に文字列`"Yes"`を出力し、改行する
3. else句を使って、`cout`に文字列`"No"`を出力し、改行する

</details><br>

**入力データ例（１）**

```txt
アーサー
```

**出力例（１）**

```txt
Yes
```

**入力データ例（２）**

```txt
ガラハド
```

**出力例（２）**

```txt
No
```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> data = {
    "アーサー",
    "ランスロット",
    "パーシヴァル",
    "ガウェイン",
    "トリスタン",
    "ケイ",
    "ガラハッド", };
  // ---- ここから上は変更しない ----

  // この下に、文字列を検索した結果を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、検索する文字列を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "Yes\nNo\nYes\nNo") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "ガラハッド" | ./practice_01a && echo "マーリン" | ./practice_01a && echo "パーシヴァル" | ./practice_01a && echo "アーサ" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> data = {
    "アーサー",
    "ランスロット",
    "パーシヴァル",
    "ガウェイン",
    "トリスタン",
    "ケイ",
    "ガラハッド", };
  // ---- ここから上は変更しない ----

  // この下に、文字列を検索した結果を出力するプログラムを書く
  string s;
  cin >> s;

  auto itr = data.find(s);
  if (itr != data.end()) {
    cout << "Yes" << endl;
  } else {
    cout << "No" << endl;
  }
}

### ❓️問題２ データベースの作成と検索

N個の文字列 $ A_1 $ ～ $ A_N $ と、検索文字列Sが入力されます。<br>
`unordered_set`型を使って、Sが $ A_1 $ ～ $ A_N $ のいずれかと一致する場合は`"Yes"`、どれにも一致しない場合は`"No"`を出力するプログラムを作成しなさい。

**入力データ形式**

```txt
N
A1 A2 … AN
S
```

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `int`型の変数`n`を宣言し、`cin`から`n`に文字列の数を読み込む
2. `unordered_set<string>`型の変数`data`を宣言する
3. for文を使って、次の処理を`n`回実行する
    1. `string`型の変数`a`を宣言する
    2. `cin`から`a`に文字列を読み込む
    3. `emplace`関数を使って、`a`を連想配列`data`に追加する
4. `auto`型の変数`itr`を宣言し、`find`関数を使って`data`から`s`を検索した戻り値で、変数`itr`を初期化する
5. if文を使って、変数`itr`が`data.end()`と異なる場合は`cout`に文字列`"Yes"`を出力し、改行する
6. else句を使って、`cout`に文字列`"No"`を出力し、改行する

</details><br>

**入力データ例（１）**

```txt
4
red green blue yellow
blue
```

**出力例（１）**

```txt
Yes
```

**入力データ例（２）**

```txt
7
abc defg hijkl m no pqr s
def
```

**出力例（２）**

```txt
No
```


In [ ]:
%%writefile practice_01b.cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  // この下に、文字列リストから文字列を検索した結果を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp && echo "この下をクリックして、文字列の数と文字列のリスト、検索文字列を入力" && ./practice_01b

In [ ]:
# @title 実行
!diff -Z <(echo -e "Yes\nNo\nYes") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp && echo "4 red green blue yellow blue" | ./practice_01b && echo "7 abc defg hijkl m no pqr s def" | ./practice_01b && echo "1 unordered_set unordered_set" | ./practice_01b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01b.cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  // この下に、文字列リストから文字列を検索した結果を出力するプログラムを書く
  int n;
  cin >> n;

  unordered_set<string> data;
  for (int i = 0; i < n; i++) {
    string a;
    cin >> a;
    data.emplace(a);
  }

  string s;
  cin >> s;

  auto itr = data.find(s);
  if (itr != data.end()) {
    cout << "Yes" << endl;
  } else {
    cout << "No" << endl;
  }
}

### ❓️問題３ あと何人？

ある施設では、利用者IDカードを入館時と退館時に提示してもらうことで、館内にいる利用者を管理するシステムを運用しています。<br>
ある日の入館・退館データ数Nと、入館か退館かを識別する文字 $ A_1 $ ～ $ A_N $ 、 利用者ID $ B_1 $ ～ $ B_N $ が入力されます。<br>
`unordered_set`を使って、すべてのデータを処理した後、まだ館内に残っている利用者の人数を出力するプログラムを作成しなさい。

識別文字は、入館が`i`、退館が`o`で表されます。利用者IDは`1`以上の整数で表されます。

**入力データ形式**

```txt
N
A1 B1
A2 B2
︙
AN BN
```

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

* 入館時に`emplace`し、退館時に`erase`する

</details>
<br>
<details><summary>🗝️(クリックでプログラム例を見る)</summary>

**プログラム例**

1. `int`型の変数`n`を宣言し、`cin`から`n`に入退館データ数を読み込む
2. `unordered_set<int>`型の変数`data`を宣言する
3. for文を使って、次の処理を`n`回繰り返す
    1. `char`型の変数`c`を宣言する
    2. `int`型の変数`id`を宣言する
    3. `cin`から変数`c`に、識別文字を読み込む
    4. `cin`から変数`id`に、利用者IDを読み込む
    5. if文を使って、変数`c`が文字`'i'`なら、連想配列`data`に、`emplace`関数を使って変数`id`の要素を追加する
    6. else句を使って、連想配列`data`に、`erase`関数を使って変数`id`の要素を削除する
4. `cout`に、連想配列`data`のサイズを出力し、改行する

</details><br>

**入力データ例（１）**

```txt
5
i 142
i 305
o 305
i 74
o 142
```

**出力例（１）**

```txt
1
```

**入力データ例（２）**

```txt
3
i 1
i 2
i 3
```

**出力例（２）**

```txt
3
```

**入力データ例（３）**

```txt
4
i 490012
i 227103
o 490012
o 227103
```

**出力例（３）**

```txt
0
```


In [ ]:
%%writefile practice_01c.cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  // この下に、まだ館内に残っている利用者の人数を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01c practice_01c.cpp && echo "この下をクリックして、入退館データを入力" && ./practice_01c

In [ ]:
# @title 実行
!diff -Z <(echo -e "1\n3\n0") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01c practice_01c.cpp && echo "5 i 142 i 305 o 305 i 74 o 142" | ./practice_01c && echo "3 i 1 i 2 i 3" | ./practice_01c && echo "4 i 490012 i 227103 o 490012 o 227103" | ./practice_01c) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01c.cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  // この下に、まだ館内に残っている利用者の人数を出力するプログラムを書く
  int n;
  cin >> n;

  unordered_set<int> data;
  for (int i = 0; i < n; i++) {
    char c;
    int id;
    cin >> c >> id;

    if (c == 'i') {
      data.emplace(id);
    } else {
      data.erase(id);
    }
  }

  cout << data.size() << endl;
}

### ❓️問題４ 道路は一本

架空市では、市内の町をつなぐ道路の建設計画を立てています。<br>
計画中の道路はN本あり、$ i $ 番目の計画は町 $ A_i $ と町 $ B_i $ をつなぎます。

建設費用を減らすため、同じ組み合わせの町をつなぐ道路の計画が2つ以上ある場合、ひとつだけを承認し、残りの計画は中止することにしました。`unordered_set`を使って、中止する計画の数を求めるプログラムを作成しなさい。

なお、町の名前はすべて英小文字であらわされます。それ以外の文字は使われません。

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

* 2つの文字列を1つに連結してキーにする
* 連結するとき、2つの文字列のあいだに区切り文字を入れる(ab c と a bc を区別するため)
* 「辞書順」で連結する(a b と b a を同じキーにするため)

</details>
<br>
<details><summary>🗝️(クリックでプログラム例を見る)</summary>

**プログラム例**

1. `int`型の変数`n`を宣言し、`cin`から`n`に計画の数を読み込む
2. `unordered_set<string>`型の変数`plans`を宣言する
3. `int`型の変数`count`を宣言し、`0`で初期化する
4. for文を使って、次の処理を`n`回実行する
    1. `string`型の変数`a`、`b`を宣言する
    2. `cin`から`a`と`b`に町の名前を読み込む
    3. `a`が`b`より辞書順で大きい場合、`a`と`b`の値を入れ替える
    4. `string`型の変数`key`を宣言し、`a + "+" + b`で初期化する<br>(文字`+`の部分は、町名に含まれない文字ならなんでもよい)
    5. `find`メンバ関数を使って、`plans`から`key`を検索する
    5. if文を使って、`a + b`が見つからなかった場合は、次の処理を行う
        * `eplamce`メンバ関数を使って`plans`に`key`を追加する
    6. else句を使って、`count`に1を足す
5. `cout`に`count`を出力し、改行する

</details><br>

**入力データ例（１）**

```txt
5
aaa bbb
bbb ccc
ccc aaa
bbb aaa
ccc bbb
```

**出力例（１）**

```txt
2
```

**入力データ例（２）**

```txt
8
x d
x y
d c
x d
b y
c b
y b
c d
```

**出力例（２）**

```txt
3
```


In [ ]:
%%writefile practice_01d.cpp
#include <iostream>
#include <unordered_set>
#include <string>
#include <algorithm>
using namespace std;

int main() {
  // この下に、中止する計画の数を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01d practice_01d.cpp && echo "この下をクリックして、計画の数と、２つの町のリストを入力" && ./practice_01d

In [ ]:
# @title 実行
!diff -Z <(echo -e "3\n2\n0") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01d practice_01d.cpp && echo "8 x d x y d c x d b y c b y b c d" | ./practice_01d && echo "5 aaa bbb bbb ccc ccc aaa bbb aaa ccc bbb" | ./practice_01d && echo "1 unordered_set unordered_map" | ./practice_01d) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01d.cpp
#include <iostream>
#include <unordered_set>
#include <string>
#include <algorithm>
using namespace std;

int main() {
  // この下に、中止する計画の数を出力するプログラムを書く
  int n;
  cin >> n;

  unordered_set<string> plans;
  int count = 0;

  for (int i = 0; i < n; i++) {
    string a, b;
    cin >> a >> b;

    if (a > b) {
      swap(a, b);
    }

    const string key = a + "+" + b;

    if (plans.find(key) == plans.end()) {
      plans.emplace(key);
    } else {
      count++;
    }
  }

  cout << count << endl;
}

### ❓️問題５ 本の著者

プログラミングに関する本の著者を調べるデータベースを作っています。

本の名前Sが入力されます。
データベースに本Sが登録されていたら、本Sの著者を出力するプログラムを作成しなさい。<br>
ただし、本Sが登録されていない場合は、本の名前Sと、文字列`"は未登録です"`を続けて出力すること。

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `string`型の変数`s`を宣言し、`cin`から`s`に本の名前を読み込む
2. `auto`型の変数`itr`を宣言し、`find`関数を使って連想配列`data`から`s`を検索し、その戻り値で変数`itr`を初期化する
3. if文を使って、`itr`が`data.end()`と異なる場合、`cout`に`itr`が指す著者を出力し、改行する
4. else句を使って、`cout`に本の名前`s`と文字列`"は未登録です"`を出力し、改行する

</details><br>

**入力データ例（１）**

```txt
熊とワルツを
```

**出力例（１）**

```txt
トム・デマルコ
```

**入力データ例（２）**

```txt
戦うプログラマー
```

**出力例（２）**

```txt
戦うプログラマーは未登録です
```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  // キー:本の名前 データ:著者
  unordered_map<string, string> data = {
    { "C++の設計と進化", "ビャーネ・ストラウストラップ"  },
    { "熊とワルツを", "トム・デマルコ" },
    { "ハッカーと画家", "ポール・グラハム" },
    { "達人プログラマー", "アンドリュー・ハント" },
    { "人月の神話", "フレデリック・ブルックス" },
    { "アジャイル・サムライ", "ジョナサン・ラスムソン" },
  };
  // ---- ここから上は変更しない ----

  // この下に、本の著者を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、本の名前を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "ポール・グラハム\n型システム入門は未登録です\nフレデリック・ブルックス") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "ハッカーと画家" | ./practice_02a && echo "型システム入門" | ./practice_02a && echo "人月の神話" | ./practice_02a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, string> data = {
    { "C++の設計と進化", "ビャーネ・ストラウストラップ"  },
    { "熊とワルツを", "トム・デマルコ" },
    { "ハッカーと画家", "ポール・グラハム" },
    { "達人プログラマー", "アンドリュー・ハント" },
    { "人月の神話", "フレデリック・ブルックス" },
    { "アジャイル・サムライ", "ジョナサン・ラスムソン" },
  };
  // ---- ここから上は変更しない ----

  // この下に、本の著者を出力するプログラムを書く
  string s;
  cin >> s;

  auto itr = data.find(s);
  if (itr != data.end()) {
    cout << itr->second << endl;
  } else {
    cout << s << "は未登録です" << endl;
  }
}

### ❓️問題６ 得点データベース

テストの成績データベースを作りたいです。<br>
このテストにはN人の受験者がいて、受験者名は $ A_1 $ ～ $ A_N $ 、点数は $ B_1 $ ～ $ B_N $ で表されます。<br>
受験者のデータと、問い合わせる名前Sが入力されます。<br>
`unordered_map`型を使って、Sがデータに存在する場合はその人の点数を出力、存在しない場合は`Sは未登録です`と出力するプログラムを作成しなさい。

**入力データ形式**

```txt
N
A1 B1
A2 B2
 ︙
AN BN
S
```

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `int`型の変数`n`を宣言し、`cin`から`n`にデータ数を読み込む
2. `unordered_map<string, int>`型の変数`data`を宣言する
3. for文を使って、次の処理を`n`回繰り返す
    1. `string`型の変数`a`を宣言する
    2. `int`型の変数`b`を宣言する
    3. `cin`から変数`a`と`b`に、受験者名と点数を読み込む
    4. `try_emplace`関数を使って、連想配列`data`に、`a`をキー、`b`をデータとして要素を追加する。
4. `string`型の変数`s`を宣言し、`cin`から`s`に問い合わせる名前を読み込む
5. `auto`型の変数`itr`を宣言し、`find`関数を使って連想配列`data`から`s`を検索した戻り値で、変数`itr`を初期化する
6. if文を使って、`itr`が`data.end()`と異なるなら、`cout`に`i`が指す点数データを出力し、改行する
7. else句を使って、`cout`に問い合わせる名前`s`と文字列`"は未登録です"`を出力し、改行する

</details><br>

**入力データ例（１）**

```txt
5
alice 85
iona 62
uriel 44
ender 100
orgie 78
iona
```

**出力例（１）**

```txt
63
```

**入力データ例（２）**

```txt
3
aaa 10
bbb 13
ccc 8
ddd
```

**出力例（２）**

```txt
dddは未登録です
```


In [ ]:
%%writefile practice_02b.cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  // この下に、得点データベースを作成し、問い合わせた名前に対応する点数を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02b practice_02b.cpp && echo "この下をクリックして、受験人数、受験者の名前と点数のリスト、問い合わせる名前を入力" && ./practice_02b

In [ ]:
# @title 実行
!diff -Z <(echo -e "62\ndddは未登録です\n100000") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02b practice_02b.cpp && echo "5 alice 85 iona 62 uriel 44 ender 100 orgie 78 iona" | ./practice_02b && echo "3 aaa 1 bbb 2 ccc 3 ddd" | ./practice_02b && echo "1 test 100000 test" | ./practice_02b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02b.cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  // この下に、得点データベースを作成し、問い合わせた名前に対応する点数を出力するプログラムを書く
  int n;
  cin >> n;

  unordered_map<string, int> data;
  for (int i = 0; i < n; i++) {
    string a;
    int b;
    cin >> a >> b;
    data.try_emplace(a, b);
  }

  string s;
  cin >> s;
  auto itr = data.find(s);
  if (itr != data.end()) {
    cout << itr->second << endl;
  } else {
    cout << s << "は未登録です" << endl;
  }
}

### ❓️問題７ アルバイトの代金

ボッタクル商店ではN人のアルバイトを雇用しており、各人の名前は $ A_1 $ ～ $ A_N $ 、時給は $ B_1 $ ～ $ B_N $ となっています。<br>
ある日に働いたアルバイトの人数M、アルバイトの名前 $ C_1 $ ～ $ C_M $ 、労働時間 $ D_1 $ ～ $ D_M $ が入力されます。<br>
`unordered_map`型を使って、ボッタクル商店が支払う給料の総額を出力するプログラムを作成しなさい。

**入力データ形式**

```txt
N
A1 B1
A2 B2
 ︙
AN BN
M
C1 D1
C2 D2
 ︙
CM DM
```

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `int`型の変数`n`を宣言し、`cin`から`n`に雇用人数を読み込む
2. `unordered_map<string, int>`型の変数`data`を宣言する
3. for文を使って、以下の処理を`n`回繰り返す
    1. `string`型の変数`a`を宣言する
    2. `int`型の変数`b`を宣言する
    3. `cin`から変数`a`と`b`に、アルバイトの名前と時給を読み込む
    4. `try_emplace`関数を使って、連想配列`data`に、`a`をキー、`b`をデータとして要素を追加する
4. `int`型の変数`m`を宣言し、`cin`から`m`に働いた人数を読み込む
5. `int`型の変数`payments`(ペイメンツ、「支払い」という意味)を宣言し、整数`0`で初期化する
6. for文を使って、以下の処理を`n`回繰り返す
    1. `string`型の変数`c`を宣言する
    2. `int`型の変数`d`を宣言する
    3. `cin`から変数`c`と`d`に、名前と労働時間を読み込む
    4. `auto`型の変数`itr`を宣言し、次の値で初期化する
        * `find`関数を使って、連想配列`data`から`c`を検索した戻り値
    5. if文を使って、変数`itr`が`data.end()`と異なる場合、次の処理を行う
        1. `itr`が指す時給に変数`d`を掛けた値を、変数`payments`に足す
7. `cout`に変数`payments`を出力し、改行する

</details><br>

**入力データ例（１）**

```txt
3
aaa 1500
bbb 1700
ccc 1400
2
bbb 7
ccc 6
```

**出力例（１）**

```txt
20300
```

**入力データ例（２）**

```txt
5
赤井 1300
緑川 1450
青木 1280
金城 1500
銀水 1500
4
緑川 8
金城 5
赤井 6
青木 3
```

**出力例（２）**

```txt
30740
```

**入力データ例（３）**

```txt
1
one 3000
1
one 5
```

**出力例（３）**

```txt
15000
```


In [ ]:
%%writefile practice_02c.cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  // この下に、バイト料の支払総額を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02c practice_02c.cpp && echo "この下をクリックして、アルバイトの登録人数、名前と時給のリスト、アルバイトに来た人数、名前と労働時間のリストを入力" && ./practice_02c

In [ ]:
# @title 実行
!diff -Z <(echo -e "20300\n30740\n15000") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02c practice_02c.cpp && echo "3 aaa 1500 bbb 1700 ccc 1400 2 bbb 7 ccc 6" | ./practice_02c && echo "5 赤井 1300 緑川 1450 青木 1280 金城 1500 銀水 1500 4 緑川 8 金城 5 赤井 6 青木 3" | ./practice_02c && echo "1 one 3000 1 one 5" | ./practice_02c) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02c.cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  // この下に、バイト料の支払総額を出力するプログラムを書く
  int n;
  cin >> n;

  unordered_map<string, int> data;
  for (int i = 0; i < n; i++) {
    string a;
    int b;
    cin >> a >> b;
    data.try_emplace(a, b);
  }

  int m;
  cin >> m;

  int payments = 0;
  for (int i = 0; i < m; i++) {
    string c;
    int d;
    cin >> c >> d;
    auto itr = data.find(c);
    if (itr != data.end()) {
      payments += itr->second * d;
    }
  }
  cout << payments << endl;
}

### ❓️問題８ 敵との遭遇

直線上のマップにいくつかの敵が配置されています。<br>
敵の数はN体で、それぞれマップの座標 $ A_1 $ ～ $ A_N $ に配置され、各敵の戦力値は $ B_1 $～ $ B_N $ で表されます。<br>
各座標の敵は最大1体で、2体以上の敵が同じ座標に配置されることはありません。

プレイヤーは最初は座標Sにいて、座標Tに移動しようとしています。<br>
座標Tに到着するまでに遭遇する敵の戦力値を合計し、合計値を出力するプログラムを作成しなさい。<br>
一度も敵と遭遇しない場合は`0`を出力すること。

なお、座標Sおよび座標Tに存在する敵も、遭遇する敵に含めるものとします。

**入力データ形式**

```txt
N
A1 B1
A2 B2
︙
AN BN
S T
```

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `int`型の変数`n`を宣言し、`cin`から`n`に敵の数を読み込む
2. `unordered_map<int, int>`型の変数`data`を宣言する<br>キーは座標、データは戦力値とする
3. for文を使って、以下の処理を`n`回繰り返す
    1. `int`型の変数`a`と`b`を宣言する
    2. `cin`から変数`a`と`b`に、敵の座標と戦力値を読み込む
    3. `try_emplace`関数を使って、連想配列`data`に、`a`をキー、`b`をデータとして要素を追加する
4. `int`型の変数`s`と`t`を宣言し、`cin`から`s`と`t`に、プレイヤーの初期座標と移動先座標を読み込む
5. if文を使って、変数`s`が`t`より大きい場合、`swap`関数を使って`s`と`t`の値を入れ替える
6. 戦力値の合計をあらわす`int`型の変数`power`を宣言し、整数`0`で初期化する
7. for文を使って、ループ変数`i`を`s`で初期化し、`i`が`t`以下である限り、次の処理を繰り返す
    1. `auto`型の変数`itr`を宣言し、以下の値で初期化する
        * 連想配列`data`から、`find`関数を使って`i`を検索した戻り値
    2. if文を使って、`itr`が`data.end()`と異なる場合、変数`power`に`itr`が指す戦力値を足す
8. `cout`に変数`power`を出力し、改行する

</details><br>

**入力データ例（１）**

```txt
5
3 50
10 10
8 23
17 30
11 20
5 11
```

**出力例（１）**

```txt
53
```

<img width="500px" hspace="0px" src="https://raw.githubusercontent.com/tn-mai/cpp2025/refs/heads/main/images/sparse_array_1d.png" />

この入力データでは、上図のように 3, 8, 10, 11, 17 の5つの座標に敵が配置されます。<br>
プレイヤーの初期位置(始点)は座標5で、そこから終点の座標11まで移動します。<br>
このルートでは3体の敵と遭遇し、戦力値はそれぞれ 23, 10, 20 です。そのため、戦力値の合計は 53 になります。

**入力データ例（２）**

```txt
3
1 2
3 4
5 6
3 1
```

**出力例（２）**

```txt
6
```

座標が小さくなる方向に移動する場合もあります。

**入力データ例（３）**

```txt
2
100 30
1000 80
1 2
```

**出力例（３）**

```txt
0
```

一度も敵と遭遇しない場合は`0`を出力します。


In [ ]:
%%writefile practice_02d.cpp
#include <iostream>
#include <unordered_map>
#include <algorithm>
using namespace std;

int main() {
  // この下に、移動中に遭遇する敵の戦力値の合計をを出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02d practice_02d.cpp && echo "この下をクリックして、敵の数、敵の座標と戦力値のリスト、プレイヤーの始点と終点を入力" && ./practice_02d

In [ ]:
# @title 実行
!diff -Z <(echo -e "53\n0\n6") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02d practice_02d.cpp && echo "5 3 50 10 10 8 23 17 30 11 20 5 11" | ./practice_02d && echo "2 100 30 1000 80 1 2" | ./practice_02d && echo "3 1 2 3 4 5 6 3 1" | ./practice_02d) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02d.cpp
#include <iostream>
#include <unordered_map>
#include <algorithm>
using namespace std;

int main() {
  // この下に、移動中に遭遇する敵の戦力値の合計を出力するプログラムを書く
  int n;
  cin >> n;

  unordered_map<int, int> data;
  for (int i = 0; i < n; i++) {
    int a, b;
    cin >> a >> b;
    data.try_emplace(a, b);
  }

  int s, t;
  cin >> s >> t;
  if (s > t) {
    swap(s, t);
  }

  int power = 0;
  for (int i = s; i <= t; i++) {
    auto itr = data.find(i);
    if (itr != data.end()) {
      power += itr->second;
    }
  }
  cout << power << endl;
}

### ❓️問題９ 在庫管理

フィフォ商店では、商品を「先入れ先出し法」(先に仕入れた商品を先に売る方式)で管理しています。<br>
N個の仕入れと販売のデータがあります。仕入れは文字`i`と製造番号X、販売は文字`o`であらわされます。

`list`型を使って、データをすべて処理したあと、在庫に残っている商品の製造番号を、仕入れ順に出力するプログラムを作成しなさい。

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `int`型の変数`n`を宣言し、`cin`から`n`にデータ数を読み込む
2. `list<int>`型の変数`data`を宣言する
3. for文を使って、以下の処理を`n`回繰り返す
    1. `char`型の変数`c`を宣言し、`cin`から`c`にデータの種類をあらわす文字を読み込む
    2. if文を使って、変数`c`が`i`なら次の処理を行う
        1. `int`型の変数`x`を宣言し、`cin`から`x`に製造番号を読み込む
        2. `push_back`関数を使って、変数`data`の末尾に`x`を追加する
    3. else句を使って、次の処理を行う
        1. `pop_front`関数を使って、変数`data`から先頭の要素を削除する
4. for文を使って、`data.begin()`から`data.end()`までのすべての要素を`cout`に出力する<br>このとき、各要素の直後に1個の空白を出力すること
5. `cout`に改行を出力する

</details><br>

**入力データ例（１）**

```txt
6
i 125
i 127
o
i 131
i 132
o
```

**出力例（１）**

```txt
131 132
```

$
\hspace{20pt}
\boxed{\begin{alignat}{5}
i: &  & \boxed{125}       & ←125 \\
i: &  & \boxed{125, 127}    & ←127 \\
o: & \space 125← & \boxed{127}       & \\
i: &  & \boxed{127, 131}    & ←131 \\
i: &  & \boxed{127, 131, 132} & ←132 \\
o: & \space 127← & \boxed{131, 132}    & \\
\end{alignat}}
$

1. 最初の`i`(仕入れ)によって、製造番号`125`の商品が在庫になります。
2. 次の`i`では製造番号`127`の商品が追加され、在庫は`125, 127`となります。
3. 次の`o`(販売)によって、最初に仕入れた商品が売られて、在庫は`127`になります。
4. 次の`i`で製造番号`131`の商品が追加され、在庫は`127, 131`になります。
5. 次の`i`で製造番号`132`の商品が追加され、在庫は`127, 131, 132`になります。
6. 次の`o`によって先に仕入れた商品が売られ、在庫は`131, 132`になります。

**入力データ例（２）**

```txt
4
i 2
i 5
o
o
```

**出力例（２）**

```txt

```

在庫がない場合は改行だけ出力します。

**入力データ例（３）**

```txt
7
i 11
i 15
o
i 22
o
i 19
i 22
```

**出力例（３）**

```txt
22 19 22
```


In [ ]:
%%writefile practice_03a.cpp
#include <iostream>
#include <list>
#include <string>
using namespace std;

int main() {
  // この下に、在庫に残った商品の製造番号を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_03a practice_03a.cpp && echo "この下をクリックして、仕入れと販売のデータを入力" && ./practice_03a

In [ ]:
# @title 実行
!diff -Z <(echo -e "22 19 22\n\n131 132") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_03a practice_03a.cpp && echo "7 i 11  i 15 o i 22 o i 19 i 22" | ./practice_03a && echo "4 i 2 i 5 o o" | ./practice_03a && echo "6 i 125 i 127 o i 131 i 132 o" | ./practice_03a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_03a.cpp
#include <iostream>
#include <list>
#include <string>
using namespace std;

int main() {
  // この下に、在庫に残った商品の製造番号を出力するプログラムを書く
  int n;
  cin >> n;

  list<int> data;
  for (int i = 0; i < n; i++) {
    char a;
    cin >> a;

    if (a == 'i') {
      int b;
      cin >> b;
      data.emplace_back(b);
    } else {
      data.pop_front();
    }
  }

  for (auto i = data.begin(); i != data.end(); i++) {
    cout << *i << ' ';
  }
  cout << endl;
}

### ❓️問題１０ プレイリスト

ヒバリさんは、自分の音楽のプレイリストにいくつかの操作をして、プレイリストの内容を変更しました。<br>
ヒバリさんの操作前のプレイリストの曲数N、曲名 $ A_1 $ ～ $ A_N $ 、操作の回数M、操作内容 $ B_1 $ ～ $ B_M $ が入力されます。<br>
すべての操作を実行した後の、プレイリストの内容を出力するプログラムを作成しなさい。

なお、操作内容Bは次の2種類のいずれかであらわされます。

* `a 曲名A 曲名B` : 曲の追加。「曲名A」の直前に「曲名B」を追加する。
* `b 曲名A` : 曲の削除。「曲名A」をリストから削除する。

**入力データ形式**

```txt
N
A1 A2 … AN
M
B1
B2
⋮
BN
```

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `int`型の変数`n`を宣言し、`cin`から`n`に最初の曲数を読み込む
2. `list<string>`型の変数`data`を宣言する
3. for文を使って、次の処理を`n`回繰り返す
    1. `string`型の変数`a`を宣言し、`cin`から`a`に曲名を読み込む
    2. `push_back`関数を使って、`data`の末尾に`a`を追加する
4. `int`型の変数`m`を宣言し、`cin`から`m`に操作回数を読み込む
5. for文を使って、次の処理を`n`回繰り返す
    1. `char`型の変数`c`を宣言する
    2. `string`型の変数`a`を宣言する
    3. `cin`から変数`c`と`a`に、「操作の種類」と「曲名A」を読み込む
    4. `auto`型の変数`itr`を宣言し、`find`関数を使って`data`から`a`を検索した結果で初期化する
    5. if文を使って、`c`が`a`の場合に次の処理を行う
        1. `string`型の変数`b`を宣言し、`cin`から`b`に「曲名B」を読み込む
        2. `insert`関数を使って、`itr`の直前に`b`を追加する
    6. else句を使って、次の処理を行う
        1. `erase`関数を使って、`itr`が指す要素を削除する
6. for文を使って、`data.begin()`から`data.end()`の範囲について、次の処理を行う
    1. `cout`に、イテレータが指す要素を出力する
    2. `cout`に1個の空白を出力する
7. `cout`に改行を出力する

</details><br>

**入力データ例（１）**

```txt
5
aaa bbb ccc ddd eee
4
a ccc fff
b bbb
b eee
a fff ggg
```

$
\hspace{20pt}
\boxed{\hspace{20pt} \begin{align}
初期状態　 & \boxed{ aaa ↔ bbb ↔ ccc ↔ ddd ↔ eee } \\
cccの直前にfffを追加　 & \boxed{ aaa ↔ bbb ↔ \boxed{\color{lightgreen}{fff}} ↔ ccc ↔ ddd ↔ eee } \\
bbbを削除　 & \boxed{ aaa ← } ... \color{red}{bbb} ... \boxed{ → fff ↔ ccc ↔ ddd ↔ eee　　 } \\
eeeを削除　 & \boxed{ aaa ↔ fff ↔ ccc ↔ ddd } ... {\color{red}{eee}} \\
fffの直前にgggを追加　 & \boxed{ aaa ↔ \boxed{\color{lightgreen}{ggg}} ↔ fff ↔ ccc ↔ ddd }
\end{align}}
$

1. 操作`a ccc fff`によって、`ccc`の直前に`fff`を追加
2. 操作`b bbb`によって、`bbb`を削除
3. 操作`b. eee`によって、`eee`を削除
4. 操作`a fff ggg`によって、`fff`の直前に`ggg`を追加

**出力例（１）**

```txt
aaa ggg fff ccc ddd
```

**入力データ例（２）**

```txt
3
MuteCity BigBlue PortTown
3
b MuteCity
b PortTown
b BigBlue
```

**出力例（２）**

```txt

```

プレイリストが空になった場合は、改行のみを出力します。

**入力データ例（３）**

```txt
3
a b c
10
a a y
a y h
a h x
b h
a c w
a c z
b a
b w
b b
b c
```

**出力例（３）**

```txt
x y z
```


In [ ]:
%%writefile practice_03b.cpp
#include <iostream>
#include <list>
#include <string>
#include <algorithm>
using namespace std;

int main() {
  // この下に、を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_03b practice_03b.cpp && echo "この下をクリックして、を入力" && ./practice_03b

In [ ]:
# @title 実行
!diff -Z <(echo -e "aaa ggg fff ccc ddd\nx y z\n") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_03b practice_03b.cpp && echo "5 aaa bbb ccc ddd eee 4 a ccc fff b bbb b eee a fff ggg" | ./practice_03b && echo "3 a b c 10 a a y a y h a h x b h a c w a c z b a b w b b b c" | ./practice_03b && echo "3 MuteCity BigBlue PortTown 3 b MuteCity b PortTown b BigBlue" | ./practice_03b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_03b.cpp
#include <iostream>
#include <list>
#include <string>
#include <algorithm>
using namespace std;

int main() {
  // この下に、を出力するプログラムを書く
  int n;
  cin >> n;

  list<string> data;
  for (int i = 0; i < n; i++) {
    string a;
    cin >> a;
    data.push_back(a);
  }

  int m;
  cin >> m;
  for (int i = 0; i < m; i++) {
    char c;
    string a;
    cin >> c >> a;

    auto itr = find(data.begin(), data.end(), a);

    if (c == 'a') {
      string b;
      cin >> b;
      data.insert(itr, b);
    } else {
      data.erase(itr);
    }
  }

  for (auto i = data.begin(); i != data.end(); i++) {
    cout << *i << ' ';
  }
  cout << endl;
}